In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset 
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
import os
import math
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import time
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
%matplotlib notebook
%run utils.ipynb

In [2]:
hits_from_seeds = np.load('./hits_from_seeds.npy')

In [3]:
hits, cells, particles, truth = load_data_single_event(1050)

In [4]:
"""
    hits: tensor of samples x timesteps x features
    bounds: K:V of x_max, x_min, y_max, y_min, z_max, z_min
"""

"""
assuming input is samples x [x_val,y_val,z_val]
"""
def calc_norm(xyz, x_min, x_max, y_min, y_max, z_min, z_max):
    norm_x = ((xyz[0] - x_min)/(x_max - x_min))
    norm_y = ((xyz[1] - y_min)/(y_max - y_min))
    norm_z = ((xyz[2] - z_min)/(z_max - z_min))
    return [norm_x,norm_y,norm_z]

def norm_hits(hits, bounds):
    assert ('x_max' and 'x_min' and 'y_max' and 'y_min' and 'z_max' and 'z_min' in bounds) or ('r_max' and 'r_min' and 'phi_max' and 'phi_min' and 'z_max' and 'z_min' in bounds)
    return np.apply_along_axis(calc_norm, 1, hits, x_min = bounds['x_min'], x_max = bounds['x_max'], y_min = bounds['y_min'], y_max = bounds['y_max'], z_min = bounds['z_min'], z_max = bounds['z_max']) 


def calc_bounds():
    bounds={'x_max':0, 'x_min':0, 'y_max':0, 'y_min':0, 'z_max':0, 'z_min':0}
    for num in range(1000,1099):
        file_name = 'event00000' + str(num)
        hits, cells, particles, truth = load_event('data/train_sample/'+file_name)
        event_x_max, event_x_min, event_y_max, event_y_min, event_z_max, event_z_min = hits['x'].max(), hits['x'].min(), hits['y'].max(), hits['y'].min(), hits['z'].max(), hits['z'].min() 
        if bounds['x_max'] < event_x_max:
            bounds['x_max'] = event_x_max
        if bounds['x_min'] > event_x_min:
            bounds['x_min'] = event_x_min
        if bounds['y_max'] < event_y_max:
            bounds['y_max'] = event_y_max
        if bounds['y_min'] > event_y_min:
            bounds['y_min'] = event_y_min
        if bounds['z_max'] < event_z_max:
            bounds['z_max'] = event_z_max
        if bounds['z_min'] > event_y_min:
            bounds['z_min'] = event_y_min
    return bounds
        

"""
input: hits and truth dfs

output: df of particle_id as index and list of particles 
"""

def gen_tracks(truth_df):
    assert(isinstance(truth_df, pd.DataFrame))
    #print(truth_df)
    truth_df['dist'] = np.sqrt(truth_df['tx']**2+truth_df['ty']**2+truth_df['tz']**2)
    grouped = truth_df.groupby('particle_id')['hit_id','dist']
    a = grouped.apply(lambda x: x.sort_values(by=['dist'],ascending=True))
    final = a.groupby('particle_id')['hit_id'].apply(np.array)
    return final 

##there's still a problem with how to deal with hits that have a particle id of 0

def batch_iter(truth_df, batch_size):
    tracks = gen_tracks(truth_df).values
    np.random.shuffle(tracks) 
    remainder = len(tracks) % batch_size if len(tracks) % batch_size is not 0 else 0
    if remainder is not 0:
        modded_tracks = tracks[:-remainder]
    else:
        modded_tracks = tracks 
    assert(len(modded_tracks)%batch_size is 0)
    for batch in modded_tracks.reshape(-1,batch_size,1):
        yield batch
        
def get_data(max_seq_len, batch_size, feature_len, truth_df, hits_df):
    hits = hits_df
    max_seq_len = max_seq_len
    b_size = batch_size
    features = feature_len #xyz or phi r z
    all_data = list(batch_iter(truth_df,b_size))
    
    #print(all_data)
    for result in all_data:
        batch = []
        batch_lv = []
        labels_tensor = []
        labels_tensor_lv = []
        for track_list in result:
            for hit_id in track_list:
                hit_coord = []
                track = []
                track_lb = []
                lv_pair = []
                lv_tensor = []
                label_coord = []
                for elem in hit_id:
                    x, y, z, layer_id, volume_id = hits.loc[hits['hit_id']== elem]['x'].item(), hits.loc[hits['hit_id']== elem]['y'].item(), hits.loc[hits['hit_id']== elem]['z'].item(), hits.loc[hits['hit_id'] == elem]['volume_id'].item(), hits.loc[hits['hit_id'] == elem]['layer_id'].item()
                    r,phi,z = cartesian_to_3d_polar(x,y,z)
                    hit_coord = [r,phi,z,layer_id, volume_id]
                    label_coord = [r,phi,z]
                    track.append(hit_coord)
                    track_lb.append(label_coord)
                    layer, volume = hits.loc[hits['hit_id'] == elem]['volume_id'].item(), hits.loc[hits['hit_id'] == elem]['layer_id'].item()
                    lv_pair = [layer, volume]
                    lv_tensor.append(lv_pair)
                zeros_to_add = max_seq_len - len(track)
                if zeros_to_add > 0:
                    add_array = np.zeros((zeros_to_add,feature_len))
                    add_array_lb = np.zeros((zeros_to_add, 3)) #3 is hardcoded for xyz/rphiz
                    add_array_lv = np.zeros((zeros_to_add, 2))
                    np_data = np.array(track)
                    np_data_lb = np.array(track_lb)
                    np_data_lv = np.array(lv_tensor)
                    padded_track_data  = np.append(np_data,add_array,axis=0)
                
                    padded_track_data_lb = np.append(np_data_lb, add_array_lb, axis=0)
                    padded_track_data_lv = np.append(np_data_lv, add_array_lv, axis=0)
                elif zeros_to_add < 0:
                    modded_track = track[:zeros_to_add]
                    modded_track_lv = lv_tensor[:zeros_to_add]
                    modded_track_lb = track_lb[:zeros_to_add]
                    padded_track_data_lb = np.array(modded_track_lb)
                    padded_track_data = np.array(modded_track)
                    padded_track_data_lv = np.array(modded_track_lv)
                else:
                    padded_track_data_lb = np.array(track_lb)
                    padded_track_data = np.array(track)
                    padded_track_data_lv = np.array(lv_tensor)
            
            row_label = padded_track_data_lb[1:]
            row_label_lv = padded_track_data_lv[1:]
            padded_row_label = np.append(row_label, np.zeros((1,3)), axis=0) #hardcoded 3 for xyz/rphiz
            padded_row_label_lv = np.append(row_label_lv, np.zeros((1,2)), axis=0)
            labels_tensor.append(padded_row_label)
            labels_tensor_lv.append(padded_row_label_lv)
            batch.append(padded_track_data)
            batch_lv.append(padded_track_data_lv)
        padded_batch_data = np.array(batch)
        padded_batch_data_lv = np.array(batch_lv)
        padded_labels = np.array(labels_tensor)
        padded_labels_lv = np.array(labels_tensor_lv)
        #print(padded_labels)
        #print(padded_labels_lv)
        yield padded_batch_data, padded_labels, padded_batch_data_lv, padded_labels_lv
        
def next_batch(max_seq_len, batch_size, feature_len):
    all_data = load_dataset('data/train_sample/', parts=['hits','truth'])
    for data in all_data:
        hit_df, truth_df = data[1], data[2]
        yield from get_data(max_seq_len, batch_size, feature_len, truth_df, hit_df)
        
def get_lv_id(x, y, z):
    row = hits.loc[(hits['x'] == x) & (hits['y'] == y) & (hits['z'] == z)]
    return int(row.iloc[0][4]), int(row.iloc[0][5])

def next_seed(hits_from_seeds):
    for seed in hits_from_seeds:
        track_seed = seed.reshape(1,3,5)
        yield track_seed.tolist()
        
def distance_between_two_points(p1, p2):
    distance = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + (p2[2] - p1[2])**2)
    return distance

def get_xyz(hit_id):
    xyz = hits[hits["hit_id"] == hit_id].iloc[:,1:4].values.tolist()[0]
    return xyz

In [5]:
unique_lv_pairs = []
for row in hits.iloc[:,4:6].itertuples():
    lv_pair = (row[1], row[2])
    if lv_pair not in unique_lv_pairs:
        unique_lv_pairs.append(lv_pair)

fcl_dict = {}
for lv_pair in unique_lv_pairs:
    lv = str(lv_pair[0]) + "-" + str(lv_pair[1])
    inputs = tf.placeholder(tf.float32, [None, 1, 3])
    num_outputs = 30
    layer = tf.contrib.layers.fully_connected(inputs, num_outputs, activation_fn=tf.nn.relu)
    fcl_dict[lv] = layer

def get_fcl(layer_id, volume_id):
    lv_pair = str(layer_id) + "-" + str(volume_id)
    for key, value in fcl_dict.items():
        if lv_pair == key:
            return value

Instructions for updating:
Use the retry module or similar alternatives.


In [12]:
def predict_next_hit(hits_so_far):
    n_steps = len(hits_so_far[0])
    n_input_features = 5
    n_neurons = 200
    n_outputs = 3
    
    
    
    restoreGraph = tf.train.import_meta_graph('./checkpoints/baseline-idealv4/-10802.meta')
    #print_tensors_in_checkpoint_file(file_name='./checkpoints/baseline-idealv4/-10802', tensor_name='', all_tensors=True, all_tensor_names=True)
    X = tf.placeholder(tf.float32, [None, n_steps, n_input_features], name='input')
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as pred_sess:

        init.run()
        #restoreGraph.restore(lstm3sess, './checkpoints/baseline-testv2/checkpoint')
        #inputs = graph.get_tensor_by_name("input:0")
        #saver = tf.train.Saver()
        graph = tf.get_default_graph()
        #ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/baseline-idealv4/'))
        # if that checkpoint exists, restore from checkpoint
        #if ckpt and ckpt.model_checkpoint_path:
        #    saver.restore(pred_sess, ckpt.model_checkpoint_path)
        restoreGraph.restore(pred_sess, tf.train.latest_checkpoint('./checkpoints/baseline-idealv4/'))
        prediction = graph.get_tensor_by_name("print_op:0")
        x_data = hits_so_far
        predicted_hit = pred_sess.run(prediction, feed_dict={X:x_data})
        return [predicted_hit[0][-1][0], predicted_hit[0][-1][1], predicted_hit[0][-1][2]]

In [13]:
error = 100
max_hits_per_tracks = 10
seeded_hits = next_seed(hits_from_seeds)
hits_from_tracks = []
start = time.time()
for i in range(2):
    print("iteration: " + str(i))
    feed_seed = next(seeded_hits)
    count = 0
    while (len(feed_seed[0]) <= max_hits_per_tracks - 1):
        print("num predicted hits: " + str(count))
        #print("hits so far (feed_seed): ", feed_seed[0])
        predicted_hit = predict_next_hit(feed_seed)
        predicted_hit = polar_to_cartesian(predicted_hit[0], predicted_hit[1], predicted_hit[2])
        #print("predicted_hit: ", predicted_hit)
        next_hit = get_xyz (closestHit (predicted_hit[0], predicted_hit[1], predicted_hit[2], hits) )
        lv = get_lv_id(next_hit[0], next_hit[1], next_hit[2])
        next_hit.extend((lv[0], lv[1]))
        #print("next hit in sequence: ", next_hit)
        if distance_between_two_points(predicted_hit, next_hit) > error:
            break
        feed_seed[0].append(next_hit)
        #print(feed_seed[0])
        count = count + 1
    #print(feed_seed)
    hits_from_tracks.append(feed_seed)
end = time.time()
print(end - start)

iteration: 0
num predicted hits: 0
INFO:tensorflow:Restoring parameters from ./checkpoints/baseline-idealv4/-11002


InvalidArgumentError: You must feed a value for placeholder tensor 'input_ph' with dtype float and shape [?,10,5]
	 [[Node: input_ph = Placeholder[dtype=DT_FLOAT, shape=[?,10,5], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: rnn/transpose_1/_61 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_151_rnn/transpose_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input_ph', defined at:
  File "/home/dl-one/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/dl-one/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dl-one/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File 